In [4]:
import openai
from openai import OpenAI
import os  

import requests  
import json  

api_key = os.environ["OPENAI_API_KEY"]  

In [5]:
# Define the API endpoint URL  
def get_response_siemens_api(url, model, max_token, user_content, system_content):  
    # Define the request headers  
    headers = {  
        'accept': 'application/json',  
        'Authorization': 'Bearer {}'.format(api_key),  
        'Content-Type': 'application/json'  
    }  

    # Define the request payload  
    payload = {  
        'model': model,  
        'messages': [  
            {'role': 'user', 'content': user_content },
            {'role': 'system', 'content': system_content}           
        ],  
        'temperature': 0,  
        'stream': False,  
        'max_tokens': max_token  
        
    }  
      
    # Make the POST request  
    response = requests.post(url, headers=headers, data=json.dumps(payload))  
      
    # Print the response  
    return print(response.json()["choices"][0]["message"]["content"])


In [6]:
url = 'https://api.siemens.com/llm/chat/completions'  
model = 'mistral-7b-instruct'
max_token = 100
user_content = ("Please classify the given requirement into 'functional requirement' or 'non-functional requirement'. "
                "The answer should be in format {the given requirement: functional requirement or non-functional requirement}."
                "The given requirement: The system shall refresh the display every 60 seconds." )
system_content = ("You are a senior software engineer who is experienced in software requirement classification! " 
                #"Please classfy the given requirement into 'functional requirement' or 'non-functional requirement'. " 
                 )

get_response_siemens_api(url, model, max_token, user_content, system_content)

 {The system shall refresh the display every 60 seconds: functional requirement}


# load the processed csv file

## evaluate with promise

In [8]:
import csv
def load_csv(file_path):  
    data = []  
    with open(file_path, 'r') as file:  
        csv_reader = csv.reader(file)  
        for row in csv_reader:  
            data.append(row)  
    return data  

In [12]:
file_path = '/Users/yongjiantang/Desktop/tang/code/re_genai/data/processed_promise.csv'  
loaded_data = load_csv(file_path)  
loaded_data[:3]

[['RequirementText', 'Class', 'b_class'],
 ["'The system shall refresh the display every 60 seconds.'", 'PE', 'NF'],
 ["'The application shall match the color of the schema set forth by Department of Homeland Security'",
  'LF',
  'NF']]

In [ ]:
def evaluate_the_dataset(loaded_data, num_shot, method):
    """ loaded_data is a N x 3 array for promise dataset, N x 2 array for nfr_so dataset
    """
    for sample in loaded_data:
        sample[0]

In [ ]:
def prompt_construction(num_shot, method, bi_cla):
    
    prompt_base = 
    if method == "random":
        prompt = 

## evaluate with nfr so

In [ ]:
file_path = '/Users/yongjiantang/Desktop/tang/code/re_genai/data/processed_nfr_so.csv'  
loaded_data = pd.read_csv(file_path, delimiter='!#!', encoding='utf-8')  
loaded_data